In [4]:
from os import listdir
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

from sklearn.ensemble import VotingClassifier


import numpy as np

from sklearn.datasets import make_classification

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix



output_path_name = "task9.2_raquel_60.txt"

output_path = "evaluations/" + output_path_name
results_path = output_path.replace('.txt', '_All_scores.log')
datadir = '../../data/Test-DDI/DrugBank'
training_data = '/home/raquel/Documents/mai/ahlt/data/Train/All'
train_df_path = '/home/raquel/Documents/mai/ahlt/data/DF/train.csv'

import warnings
warnings.filterwarnings('ignore')

In [5]:
train_df = pd.read_csv(train_df_path, index_col=0)

sentences = train_df.sentence_text.values
y = train_df['relation_type'].values

sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=1000,stratify=y)


def vectorize_data(sentences_train,sentences_test):  
    
    vectorizer = CountVectorizer()
    vectorizer.fit(sentences_train)
    
    X_train = vectorizer.transform(sentences_train)
    X_test =  vectorizer.transform(sentences_test)
    
    return X_train, X_test

X_train, X_test = vectorize_data(sentences_train,sentences_test)

In [6]:
rf = RandomForestClassifier(n_estimators=1000, max_depth=60, n_jobs=-1,
                                class_weight='balanced', random_state=0)

mlp = MLPClassifier(activation='tanh', alpha= 0.1, hidden_layer_sizes=(30, 5), learning_rate='constant')

classifier = VotingClassifier(estimators=[('Random Forest', rf), ('MLP', mlp)],
                            voting='soft')

classifier.fit(X_train, y_train)

y_true, y_pred = y_test, classifier.predict(X_test)

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

      advise       0.29      0.49      0.37       206
      effect       0.35      0.51      0.41       422
         int       0.18      0.43      0.25        47
   mechanism       0.26      0.27      0.26       330
        none       0.90      0.84      0.87      5943

   micro avg       0.78      0.78      0.78      6948
   macro avg       0.40      0.51      0.43      6948
weighted avg       0.81      0.78      0.79      6948

